In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import models
from keras import Sequential
from keras import layers
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.metrics import AUC
from keras.regularizers import L1
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup 
import re

2024-04-14 19:12:16.216372: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-14 19:12:16.264493: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 19:12:16.264524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 19:12:16.265414: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-14 19:12:16.271360: I tensorflow/core/platform/cpu_feature_guar

In [2]:
colnames=['ID', 'Social network', 'Sentiment', 'Text'] 

In [3]:
df_train = pd.read_csv('twitter_training.csv', names=colnames, header=None)
df_val = pd.read_csv('twitter_validation.csv', names=colnames, header=None)

In [4]:
df_train.tail()

,ID,Social network,Sentiment,Text
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...
74681,9200,Nvidia,Positive,Just like the windows partition of my Mac is l...


Взял из примера предобработку

In [13]:
def cleaned_review(review):
  stop = stopwords.words("english")
  stop.remove('not')
  stop.remove('no')
  
  lemmatizer = WordNetLemmatizer()
  # убираем теги html
  new_review = BeautifulSoup(review).get_text()
  # убираем url
  no_urls = review.replace('http\S+', '').replace('www\S+', '')
  # убираем символы кроме букв
  clean_review = re.sub("[^a-zA-Z]", " ", no_urls)
  # приводим в нижнему регистру
  new_words = clean_review.lower().split()
  # лематизируем и убираем стоп-слова
  final_words = [lemmatizer.lemmatize(word) for word in new_words if not word in stop]
    
  return (" ".join(final_words))

In [14]:
df_train['Text'].head()

0    im getting on borderlands and i will murder yo...
1    I am coming to the borders and I will kill you...
2    im getting on borderlands and i will kill you ...
3    im coming on borderlands and i will murder you...
4    im getting on borderlands 2 and i will murder ...
Name: Text, dtype: object

In [17]:
df_train['Text'] = df_train['Text'].apply(cleaned_review)

/tmp/ipykernel_10407/1577662565.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  new_review = BeautifulSoup(review).get_text()


TypeError: object of type 'float' has no len()

In [16]:
df_val['Text'] = df_val['Text'].apply(cleaned_review)

/tmp/ipykernel_10407/1577662565.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  new_review = BeautifulSoup(review).get_text()


In [ ]:
train_data = df_train['Text'].to_list()
val_data = df_val['Text'].to_list()

train_data = np.concatenate([train_data, val_data])

In [ ]:
train_categ = df_train['Sentiment']
val_categ = df_val['Sentiment']

In [ ]:
labels = list(df_train['Sentiment'].unique())
labeltoid = {v:k for k,v in enumerate(labels)}

In [ ]:
train_y = np.array(df_train["Sentiment"].map(labeltoid).to_list())
val_y = np.array(df_val['Sentiment'].map(labeltoid).to_list())
train_y = np.concatenate([train_y, val_y])

In [ ]:
from transformers import AutoTokenizer, MobileBertModel

In [ ]:
token = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
model = MobileBertModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

You are using a model of type bert to instantiate a model of type mobilebert. This is not supported for all configurations of models and can yield errors.
Some weights of MobileBertModel were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.embedding_transformation.bias', 'embeddings.embedding_transformation.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'enco

In [ ]:
train_data = list(train_data)

In [ ]:
X_encoded = token(train_data, truncation=True, padding=True)

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).